In [1]:
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyreadr

%load_ext memory_profiler

In [2]:
data_src = "./data/Data_20130610.RData"

In [3]:
r_data = pyreadr.read_r(data_src)

In [4]:
data1 = r_data["Data1"]
data2 = r_data["Data2"]

## `Data1` -- misclassifications

In [5]:
data1.head()

,IdTooth,VISIT,EXAMINER,STATUS,TOOTH
0,1_16,7.690623,15,0,16
1,1_16,8.544832,8,0,16
2,1_16,9.462012,15,0,16
3,1_16,10.193018,8,0,16
4,1_16,11.170431,14,0,16


* Change STATUS to float.
* Split IdTooth into IDNR (patient ID) and TOOTH (tooth ID).
* Change their dtypes from str to int.
* Change EXAMINER to int (was an int but int32, this makes it an int64).
* Drop IdTooth.
* Reorder so that IDNR and TOOTH are the first two columns.

In [6]:
data1["STATUS"] = data1["STATUS"].astype(float)
data1[["IDNR", "TOOTH"]] = data1["IdTooth"].str.split("_", expand=True)
data1["IDNR"] = data1["IDNR"].astype(int)
data1["TOOTH"] = data1["TOOTH"].astype(int)
data1["EXAMINER"] = data1["EXAMINER"].astype(int)
data1.drop("IdTooth", axis="columns", inplace=True)
data1 = data1[["IDNR", "TOOTH", "VISIT", "EXAMINER", "STATUS"]]

In [7]:
data1.dtypes

IDNR          int64
TOOTH         int64
VISIT       float64
EXAMINER      int64
STATUS      float64
dtype: object

In [8]:
data1.head()

,IDNR,TOOTH,VISIT,EXAMINER,STATUS
0,1,16,7.690623,15,0.0
1,1,16,8.544832,8,0.0
2,1,16,9.462012,15,0.0
3,1,16,10.193018,8,0.0
4,1,16,11.170431,14,0.0


In [9]:
data1.sort_values(by=["IDNR", "TOOTH"]).equals(data1)

True

## `Data2` -- regressors

In [10]:
data2.head()

,IDNR,TOOTH,EBEG,EEND,FBEG,FEND,GIRL,STARTBR,FREQ.BR,SEAL,PLAQUE.1,PLAQUE.2,MEALS,XCEN,YCEN,FTimeImp
0,1.0,16.0,6.7,7.7,11.2,NaN,1.0,3.0,0.0,1.0,0.0,0.0,0.0,123.543184,174.995531,11.2
1,1.0,26.0,5.0,7.7,11.2,NaN,1.0,3.0,0.0,1.0,0.0,0.0,0.0,123.543184,174.995531,11.2
2,1.0,36.0,6.7,7.7,11.2,NaN,1.0,3.0,0.0,1.0,0.0,0.0,0.0,123.543184,174.995531,11.2
3,1.0,46.0,6.7,7.7,11.2,NaN,1.0,3.0,0.0,1.0,0.0,0.0,0.0,123.543184,174.995531,11.2
4,2.0,16.0,5.0,7.4,7.4,8.4,0.0,6.0,0.0,0.0,0.0,0.0,1.0,206.175102,206.617429,7.9


In [11]:
data2.isna().sum()

IDNR            0
TOOTH           0
EBEG            0
EEND            0
FBEG          663
FEND        11494
GIRL            0
STARTBR       907
FREQ.BR         0
SEAL            0
PLAQUE.1     2474
PLAQUE.2     2474
MEALS        1799
XCEN            0
YCEN            0
FTimeImp        0
dtype: int64

* Change IDNR and TOOTH to int.

In [12]:
data2["IDNR"] = data2["IDNR"].astype(int)
data2["TOOTH"] = data2["TOOTH"].astype(int)

* Change FBEG NA to $0$.
* Change FEND NA to $+\infty$.

In [13]:
data2.loc[data2["FBEG"].isna(), "FBEG"] = 0.0
data2.loc[data2["FEND"].isna(), "FEND"] = float("inf")

In [14]:
data2.dtypes

IDNR          int64
TOOTH         int64
EBEG        float64
EEND        float64
FBEG        float64
FEND        float64
GIRL        float64
STARTBR     float64
FREQ.BR     float64
SEAL        float64
PLAQUE.1    float64
PLAQUE.2    float64
MEALS       float64
XCEN        float64
YCEN        float64
FTimeImp    float64
dtype: object

In [15]:
data2.head()

,IDNR,TOOTH,EBEG,EEND,FBEG,FEND,GIRL,STARTBR,FREQ.BR,SEAL,PLAQUE.1,PLAQUE.2,MEALS,XCEN,YCEN,FTimeImp
0,1,16,6.7,7.7,11.2,inf,1.0,3.0,0.0,1.0,0.0,0.0,0.0,123.543184,174.995531,11.2
1,1,26,5.0,7.7,11.2,inf,1.0,3.0,0.0,1.0,0.0,0.0,0.0,123.543184,174.995531,11.2
2,1,36,6.7,7.7,11.2,inf,1.0,3.0,0.0,1.0,0.0,0.0,0.0,123.543184,174.995531,11.2
3,1,46,6.7,7.7,11.2,inf,1.0,3.0,0.0,1.0,0.0,0.0,0.0,123.543184,174.995531,11.2
4,2,16,5.0,7.4,7.4,8.4,0.0,6.0,0.0,0.0,0.0,0.0,1.0,206.175102,206.617429,7.9


In [16]:
data2.isna().sum()

IDNR           0
TOOTH          0
EBEG           0
EEND           0
FBEG           0
FEND           0
GIRL           0
STARTBR      907
FREQ.BR        0
SEAL           0
PLAQUE.1    2474
PLAQUE.2    2474
MEALS       1799
XCEN           0
YCEN           0
FTimeImp       0
dtype: int64

In [17]:
data2.sort_values(by=["IDNR", "TOOTH"]).equals(data2)

True

In [19]:
grouped = data2.groupby("IDNR").count()
not_4_teeth = grouped[grouped["TOOTH"] != 4].index

In [26]:
data2.drop(data2[data2["IDNR"].isin(not_4_teeth)].index).shape

(15332, 16)

## Join Data1 and Data2

In [ ]:
%memit df = pd.merge(data1, data2, how="inner", on=["IDNR", "TOOTH"], validate="many_to_one")

In [ ]:
df.memory_usage(deep=True).values.sum() / 1024 / 1024

In [ ]:
df.sort_values(by=["IDNR", "TOOTH"]).equals(df)